In [1]:
import random
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch

seed = 42
deterministic = False

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
if deterministic:
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

In [2]:
train_df = pd.read_csv("./data/train_preproc.csv").drop('ID', axis=1)
# test_df = pd.read_csv("./data/test_preproc.csv")

In [3]:
train_df

,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,12480000,36,6,0,72000000,18.90,15,4,0,0,0.0,0.0,0.0,2
1,14400000,60,10,2,130800000,22.33,21,2,0,373572,234060.0,0.0,0.0,1
2,12000000,36,5,2,96000000,8.60,14,4,0,928644,151944.0,0.0,0.0,0
3,14400000,36,8,2,132000000,15.09,15,4,0,325824,153108.0,0.0,0.0,2
4,18000000,60,-1,0,71736000,25.39,19,3,0,228540,148956.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,14400000,36,10,2,210000000,9.33,33,0,0,974580,492168.0,0.0,0.0,2
96290,28800000,60,10,2,132000000,5.16,25,2,0,583728,855084.0,0.0,0.0,4
96291,14400000,36,1,2,84000000,11.24,22,0,0,1489128,241236.0,0.0,0.0,0
96292,15600000,36,5,2,66330000,17.30,21,4,2,1378368,818076.0,0.0,0.0,3


In [4]:
x_data = train_df.drop('대출등급', axis=1)
y_data = train_df['대출등급']

In [5]:
scaler = StandardScaler()
x_data = scaler.fit_transform(x_data.iloc[:, :-1])

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, shuffle=True)
print(len(x_train), len(x_test))

77035 19259


In [ ]:
# XGB objective
    # reg:squarederror
    # reg:squaredlogerror
    # reg:logistic
    # reg:pseudohubererror
    # reg:absoluteerror
    # reg:quantileerror

# LGBM objective
    # regression
    # regression_l1
    # huber
    # fair
    # poisson
    # quantile
    # mape
    # tweedie

In [ ]:
reg_list = [
    XGBRegressor(n_estimators=100,
                 learning_rate=0.08,
                 gamma=0,
                 subsample=0.75,
                 colsample_bytree=1,
                 max_depth=7),
    LGBMRegressor(n_estimators=100,
                  learning_rate=1e-5)
]